#11.Implement a simple top-down parser for context-free grammars using python.#


In [ ]:
class SimpleParser:
    def __init__ (self, grammar):
        self.grammar = grammar
    def parse (self, input_string):
        self.input = input_string
        self.index = 0
        self.result = True
        if self.expression ():
            if self.index == len (self.input):
                print (f'Parsing successful for input: {input_string}')
                return
        print (f'Parsing failed for input: {input_string}')
    def expression (self):
        return self.term () and self.expression_tail ()
    def expression_tail (self):
        current_index = self.index
        if self.match ('+'):
            return self.term () and self.expression_tail ()
        self.index = current_index
        return True
    def term (self):
        return self.factor () and self.term_tail ()
    def term_tail (self):
        current_index = self.index
        if self.match ('*'):
            return self.factor () and self.term_tail ()
        self.index = current_index
        return True
    def factor (self):
        if self.match ('('):
            if self.expression () and self.match (')'):
                return True
            return False
        return self.match ('number')
    def match (self, expected):
        if self.index < len (self.input) and (expected == self.input [self.index] or expected == 'number' and self.input [self.index].isdigit ()):
            self.index += 1
            return True
        return False

grammar = {
    'start': 'Expression',
}

parser = SimpleParser (grammar)

parser.parse ('3* (2+1)')
parser.parse ('2+1*3')
parser.parse ('2+ (1*1)')


Parsing failed for input: 3* (2+1)
Parsing successful for input: 2+1*3
Parsing failed for input: 2+ (1*1)


#12. Implement an Earley parser for context-free grammars using a simple python program.#


In [ ]:
class EarleyParser:
    def __init__(self, grammar):
        self.grammar = grammar
    def parse(self, input_string):
        self.chart = [[] for _ in range(len(input_string) + 1)]
        self.chart[0].append(('start', '', 0))
        for i in range(len(input_string) + 1):
            for state in self.chart[i]:
                self.predictor(state, i)
                if i < len(input_string):
                    self.scanner(state, input_string[i], i)
                else:
                    self.completer(state, i)
        if ('start', self.grammar['start'], 0) in self.chart[len(input_string)]:
            print(f'Parsing failed for input: {input_string}')
        else:
            print(f'Parsing successfull for input: {input_string}')
    def predictor(self, state, index):
        if state[1] in self.grammar:
            for production in self.grammar[state[1]]:
                self.chart[index].append((state[1], production, index))
    def scanner(self, state, token, index):
        if state[1] == '' or state[1][0] != token:
            return
        self.chart[index + 1].append((state[0], state[1][1:], state[2]))
    def completer(self, state, index):
        for st in self.chart[state[2]]:
            if st[1] == '' or st[1][0] != state[0]:
                continue
            self.chart[index].append((st[0], st[1][1:], st[2]))
# Example usage
grammar = {
    'start': 'Expression',
    'Expression': ['Term + Expression', 'Term'],
    'Term': ['Factor * Term', 'Factor'],
    'Factor': ['( Expression )', 'number']
}

parser = EarleyParser(grammar)

# Test the parser
parser.parse('3* (2+1)') # Parsing successful for input: 3* (2+1)
parser.parse('2+1*3') # Parsing successful for input: 2+1*3
parser.parse('2+ (1*3)') # Parsing successful for input: 2+ (1*3)


Parsing successfull for input: 3* (2+1)
Parsing successfull for input: 2+1*3
Parsing successfull for input: 2+ (1*3)


#13. Generate a parse tree for a given sentence using a context-free grammar using python program.#

In [ ]:
import nltk
def generate_parse_tree(sentence, grammar):
    tokens = nltk.word_tokenize(sentence)
    parser = nltk.ChartParser(grammar)
    parse_tree = None
    for tree in parser.parse(tokens):
        parse_tree = tree
        break
    return parse_tree
example_grammar = nltk.CFG.fromstring("""
    S -> NP VP
    NP -> Det N
    VP -> V NP
    Det -> 'the' | 'a'
    N -> 'cat' | 'dog'
    V -> 'chased' | 'caught'
""")
example_sentence = "the cat chased a dog"
parse_tree = generate_parse_tree(example_sentence, example_grammar)
if parse_tree:
    parse_tree.pretty_print()
else:
    print("No parse tree found for the given sentence.")

              S               
      ________|_____           
     |              VP        
     |         _____|___       
     NP       |         NP    
  ___|___     |      ___|___   
Det      N    V    Det      N 
 |       |    |     |       |  
the     cat chased  a      dog



# 14.Create a program in python to check for agreement in sentences based on a context-free grammar's rules.#


In [ ]:
import nltk
def check_agreement(sentence):
    tagged_words = nltk.pos_tag(nltk.word_tokenize(sentence))
    subjects = [word for word, tag in tagged_words if tag.startswith('N')]
    verbs = [word for word, tag in tagged_words if tag.startswith('V')]
    if subjects and verbs:
        subject_number = 'singular' if tagged_words[0][1].startswith('NNS') else 'singular'
        verb_number = 'singular' if tagged_words[-1][1].startswith('VB') else 'plural'
        if subject_number != verb_number:
            print("Subject-verb agreement error:")
            print(f"Subjects: {subjects} ({subject_number})")
            print(f"Verbs: {verbs} ({verb_number})")
        else:
            print("Subject-verb agreement is correct.")
    else:
        print("Unable to find subjects and verbs in the sentence.")
example_sentence = "The cat catches a dog"
check_agreement(example_sentence)

Subject-verb agreement error:
Subjects: ['cat', 'dog'] (singular)
Verbs: ['catches'] (plural)


# 15.Implement a simple top-down parser for context-free grammars using python.#


In [ ]:
import nltk

nltk.download('punkt')

def pcfg_parse(sentence, pcfg_grammar):
    tokens = nltk.word_tokenize(sentence)
    parser = nltk.EarleyChartParser(pcfg_grammar)
    parse_tree = None
    for tree in parser.parse(tokens):
        parse_tree = tree
        break
    return parse_tree
pcfg_grammar = nltk.PCFG.fromstring("""
    S -> NP VP [1.0]
    NP -> Det N [0.5] | N [0.5]
    VP -> V NP [0.7] | VP PP [0.3]
    PP -> P NP [1.0]
    Det -> 'the' [0.8] | 'a' [0.2]
    N -> 'cat' [0.4] | 'dog' [0.6]
    V -> 'chased' [0.9] | 'caught' [0.1]
    P -> 'in' [0.6] | 'on' [0.4]
""")
example_sentence = "the cat chased a dog"
parse_tree = pcfg_parse(example_sentence, pcfg_grammar)
if parse_tree:
    parse_tree.pretty_print()
else:
    print("No parse tree found for the given sentence.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


              S               
      ________|_____           
     |              VP        
     |         _____|___       
     NP       |         NP    
  ___|___     |      ___|___   
Det      N    V    Det      N 
 |       |    |     |       |  
the     cat chased  a      dog



# 16 Implement a Python program using the SpaCy library to perform Named Entity Recognition (NER) on a given text.#

In [ ]:
import spacy
def perform_ner(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities
example_text = "Apple Inc. was founded by Steve Jobs in Cupertino. The iPhone was first released in 2007."
ner_results = perform_ner(example_text)
for entity, label in ner_results:
    print(f"{entity} - {label}")





Apple Inc. - ORG
Steve Jobs - PERSON
Cupertino - GPE
iPhone - ORG
first - ORDINAL
2007 - DATE
